In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report
import wandb
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.metrics import log_loss
from sklearn.metrics import hinge_loss

https://wandb.ai/smaia3/Ques2/reports/Question-2--Vmlldzo1NzU1MTMx?accessToken=uyschb1suafp5d39l9gkwwtwgrcxbt0ip600t2amd0qv8hgf8gyxn1xntrz4cy82

## for part B
### Activation and Opt
https://wandb.ai/smaia3/Ques2BB/reports/Q2-O-A--Vmlldzo1NzU1OTYx?accessToken=wqxy0nuy8gka3jmwv94e1azqyyulb4jvnj634jk6nexi0mw6567f57s5tx3n9wn3
### Learning rates, epochs and hidden layers 
https://api.wandb.ai/links/smaia3/yf6l05lx

In [2]:
class MLPClassifier:
    def __init__(self, input_size, hidden_layers, output_size, learning_rate, activation):
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.output_size = output_size
        self.learning_rate = learning_rate
        # rand initialization
        self.neurons_layer = [input_size] + hidden_layers + [output_size]  
        self.weights = [np.random.randn(self.neurons_layer[i], self.neurons_layer[i + 1]) for i in range(len(self.neurons_layer) - 1)]
        self.biases = [np.zeros((1, neurons)) for neurons in self.neurons_layer[1:]]
        self.activation = activation
        self.convegrance = [1,2,3,4,5,6,7,8,9,1,2,3,4,5] # random list to check convegrance
        self.istrain = 0

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / exp_x.sum(axis=1, keepdims=True)
        
    def sigmoid(self, x):
        if(self.activation == 'sigmoid'):
            return 1 / (1 + np.exp(-x))
        elif(self.activation == 'tanh'):
            exp_x = np.exp(2 * x)
            tanh_x = (exp_x - 1) / (exp_x + 1)
            return tanh_x
        elif(self.activation == 'relu'):
            if self.istrain:
                return np.maximum(0, 0.01*x)
            return np.maximum(0, x)

    def sigmoid_derivative(self, x):
        if(self.activation == 'sigmoid'):
            return x * (1 - x)
        elif(self.activation == 'tanh'):
            return 1 - x**2
        elif(self.activation == 'relu'):
            return (x > 0)

    def forward_propagation(self, x):
        self.layer_outputs = [x]
        for i in range(len(self.neurons_layer) - 1):
            z = np.dot(self.layer_outputs[i], self.weights[i]) + self.biases[i]
            a = self.sigmoid(z) if i < len(self.neurons_layer) - 2 else self.softmax(z) 
            self.layer_outputs.append(a)
        return self.layer_outputs[-1]

    def predict(self, X_test):
        predictions = []
        for x in X_test:
            x = x.reshape(1, -1)
            predicted = self.forward_propagation(x)
            predictions.append(predicted)
        return np.vstack(predictions)
    
    def backward_propagation(self, X, y):
        # Initialize empty lists to store gradients
        gradients = []
        deltas = [self.layer_outputs[-1] - y]
        # Calculate deltas and gradients for hidden layers and output layer
        for i in range(len(self.neurons_layer) - 2, 0, -1):
            delta = np.dot(deltas[-1], self.weights[i].T) * self.sigmoid_derivative(self.layer_outputs[i])
            deltas.append(delta)
        deltas.reverse()
        num_samples = X.shape[0]
        for i in range(len(self.hidden_layers), -1, -1):
            # Calculate gradients for weights and biases
            grad_weights = np.dot(self.layer_outputs[i].T, deltas[i]) / num_samples
            grad_biases = np.sum(deltas[i], axis=0, keepdims=True) / num_samples
            gradients.append((grad_weights, grad_biases))
        return gradients
    
    def update(self, gradients):
        gradients.reverse()
        # Update the weights and biases
        max = 0 
        for i in range(len(self.hidden_layers) + 1):
            # print shapes for debugging 
            grad_weights, grad_biases = gradients[i]
            self.weights[i] -= self.learning_rate * grad_weights
            self.biases[i] -= self.learning_rate * grad_biases
            maxx = np.max(gradients[i][0])
            if maxx > max:
                max = maxx
            maxx = np.max(gradients[i][1])
            if maxx > max:
                max = maxx
        # print(max)
        self.convegrance.append(max*self.learning_rate)

    def train(self, X_train, y_train, num_epochs):
        ep = num_epochs//10
        self.istrain = 1
        for epoch in range(ep):
            for x, y in zip(X_train, y_train):
                # mse = self.mean_squared_error(self.predict(X_train), y_train)
                # acc = self.accuracy(self.predict(X_train), y_train)
                # print("Epoch: {} | MSE: {} | Accuracy: {}".format(epoch, mse, acc))
                x = x.reshape(1, -1)
                y = y.reshape(1, -1)
                self.forward_propagation(x)
                gradients = self.backward_propagation(x, y)
                self.update(gradients)
                # check if the last 15 gradients are less than 0.0001
                # if len(self.convegrance) > 15:
                #     if np.max(self.convegrance[-15:]) < 0.01:
                #         print("Converged at epoch: ", epoch)
                #         break
            self.eval(epoch*10)
        self.istrain = 0

    def trainbatch(self, x, y, num_epochs):
        for epoch in range(num_epochs):
            gradients = []
            self.forward_propagation(x)
            gradients += self.backward_propagation(x, y)
            # given list of arrays gradients find the mean for each
            gradients = [(np.mean(grad[0], axis=0), np.mean(grad[1], axis=0)) for grad in gradients]
            self.update(gradients)
            # if epoch % 10 == 0:
            self.eval(epoch)

    def trainminibatch(self, x, y, num_epochs):
        for epoch in range(num_epochs):
            gradients = []
            # divide data into batches
            batch_size = 50
            num_batches = x.shape[0] // batch_size
            for i in range(num_batches):
                x_batch = x[i * batch_size: (i + 1) * batch_size]
                y_batch = y[i * batch_size: (i + 1) * batch_size]
                self.forward_propagation(x_batch)
                gradients += self.backward_propagation(x_batch, y_batch)
                gradients = [(np.mean(grad[0], axis=0), np.mean(grad[1], axis=0)) for grad in gradients]
                self.update(gradients)
                gradients = []
            if epoch % 10 == 0:
                self.eval(epoch)

    
    def eval(self, epoch):
        return
        predictions = self.predict(X_test)
        # Inverse transform the one-hot encoded labels to categorical
        predicted_labels = encoder.inverse_transform(predictions)
        y_test_labels = encoder.inverse_transform(y_test)
        accuracy =  accuracy_score(y_test_labels, predicted_labels)
        precision = precision_score(y_test_labels, predicted_labels, average='macro', zero_division=0)
        recall =  recall_score(y_test_labels, predicted_labels, average='macro', zero_division=0)
        f1 = f1_score(y_test_labels, predicted_labels, average='macro', zero_division=0)
        loss = log_loss(y_test, predictions)
        wandb.log({"epoch": epoch, "val_accuracy": accuracy, "val_precision": precision, "val_recall": recall, "val_f1": f1, "val_loss": loss})

        predictions = self.predict(X_train)
        predicted_labels = encoder.inverse_transform(predictions)
        y_test_labels = encoder.inverse_transform(y_train)
        train_acc = accuracy_score(y_test_labels, predicted_labels)
        train_loss = log_loss(y_train, predictions)
        wandb.log({"epoch": epoch, "train_accuracy": train_acc, "train_loss": train_loss})
        return
    
        predictions = self.predict(X_val)
        predicted_labels = encoder.inverse_transform(predictions)           
        y_test_labels = encoder.inverse_transform(y_val)     
        precision =  precision_score(y_test_labels, predicted_labels, average='macro', zero_division=0)
        recall = recall_score(y_test_labels, predicted_labels, average='macro', zero_division=0)
        f1 = f1_score(y_test_labels, predicted_labels, average='macro', zero_division=0)
        accuracy = accuracy_score(y_test_labels, predicted_labels)
        loss = log_loss(y_test_labels, predictions)
        wandb.log({"epoch": epoch, "val_accuracy": accuracy, "val_loss": loss, "val_precision": precision, "val_recall": recall, "val_f1": f1})
        # also log accuracy and loss on train set
        predictions = self.predict(X_train)
        predicted_labels = encoder.inverse_transform(predictions)
        y_test_labels = encoder.inverse_transform(y_train)
        train_acc = accuracy_score(y_test_labels, predicted_labels)
        train_loss = log_loss(y_test_labels, predictions)
        wandb.log({"epoch": epoch, "train_accuracy": train_acc, "train_loss": train_loss})

        


In [6]:
class_labels = [[3], [4], [5], [6], [7], [8]]
data = pd.read_csv('wineQT.csv')                                # Load the dataset
X = data.drop(['quality'], axis=1)                              # Extract features and labels
y = data['quality']
encoder = OneHotEncoder(sparse_output=False)                    # One-hot encode the labels
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))
scaler = StandardScaler()                                       # Scale the data
X = scaler.fit_transform(X)
X_train, xtemp, y_train, ytemp = train_test_split(X, y_encoded, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(xtemp, ytemp, test_size=0.4, random_state=42)

# iterate over various learning rates, activation functions, and hidden layers and log on WandB
learning_rates = [0.01, 0.001, 0.0001]
activations = ['tanh', 'relu', 'sigmoid']
hidden_layers = [[10], [10, 15], [10, 15, 10]]

for activation in activations:
    for lr in learning_rates:
        for hidden_layer in hidden_layers:
            wandb.init(project="Ques2", config={"learning_rate": lr, "activation": activation, "hidden_layer": hidden_layer, "Optimizer": "SGD"})
            config = wandb.config
            mlp = MLPClassifier(X_train.shape[1], config.hidden_layer, y_encoded.shape[1], config.learning_rate, config.activation)
            mlp.train(X_train, y_train, 10000)

for lr in learning_rates:
    for activation in activations:
        for hidden_layer in hidden_layers:
            wandb.init(project="Ques2", config={"learning_rate": lr, "activation": activation, "hidden_layer": hidden_layer, "Optimizer": "Batch"})
            config = wandb.config
            mlp = MLPClassifier(X_train.shape[1], config.hidden_layer, y_encoded.shape[1], config.learning_rate, config.activation)
            mlp.trainbatch(X_train, y_train, 10000)


for lr in learning_rates:
    for activation in activations:
        for hidden_layer in hidden_layers:
            wandb.init(project="Ques2", config={"learning_rate": lr, "activation": activation, "hidden_layer": hidden_layer, "Optimizer": "MiniBatch"})
            config = wandb.config
            mlp = MLPClassifier(X_train.shape[1], config.hidden_layer, y_encoded.shape[1], config.learning_rate, config.activation)
            mlp.trainminibatch(X_train, y_train, 10000)

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train_accuracy,▁▂▃▃▃▄▅▅▅▅▅▅▅▅▆▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇██████████
train_loss,█▇▆▅▅▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,█▅▆▇▆▃▄▅▄▃▄▆▅▅▆▅▆▅▄▅▄▃▃▃▃▃▃▃▃▂▂▂▄▃▃▃▃▂▂▁
val_f1,▃▂▃▅▅▂▃▄▄▄▅█▇▇▇██▇▆▇▅▂▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▃▃▂
val_loss,▁▁▁▂▂▃▃▃▄▄▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇████
val_precision,█▄▅▅▄▁▂▃▂▃▄▆▅▆▅▆▆▆▅▅▄▂▂▁▁▁▁▁▂▂▃▃▃▃▃▃▃▃▃▂
val_recall,▃▂▂▅▆▂▅▅▄▄▄█▇▇█▇█▇▆▇▅▂▁▁▃▃▂▂▄▃▃▃▅▅▅▅▄▃▃▂
epoch,5320
train_accuracy,0.74599
train_loss,0.62021


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_accuracy,▁▂▃▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█████████████
train_loss,█▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▇█▆▅▃▅▃▄▂▃▃▂▃▂▂▄▅▅▄▃▃▂▂▁▂▁▁▁▁▁▁▁▁▁▁▂▂▂▃▅
val_f1,▁▂▄▂▂▂▂▂▂▂▂▂▂▂▂▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█
val_loss,▁▁▁▂▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇█████████
val_precision,▃▂▆▁▁▁▁▁▁▁▁▁▁▁▁▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█
val_recall,▁▂▃▂▂▂▂▂▂▂▂▂▂▂▂▆▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▇█
epoch,9990
train_accuracy,0.77372
train_loss,0.58653


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


...

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_accuracy,▁▃▄▄▆▆▇███████▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▆▇▇▇▇▇▇▇▇▇▆
train_loss,█▆▅▄▄▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▄▅▆▆▇██▇███▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
val_f1,▁▂▃▃▅▅▇▇█▇▇▇█▇▇▆▆█▇▇▇▇▇▇▇▇▇▇█▇████▇████▇
val_loss,█▆▅▅▄▄▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_precision,▁▁▂▂▅▄▆▇█▆▆▇▇▆▅▅▅▇▇▇▅▅▆▆▆▆▆▇▇▇▇█▇▇▇█▇▇▇▆
val_recall,▄▁▂▃▅▅▇▇█▇▇▇█▆▆▆▆▇▇▇▇▆▇▇▇▇▇▇█▇██▇▇▇█▇██▇
epoch,9990
train_accuracy,0.41898
train_loss,2.02678


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_accuracy,▁▂▃▄▅▅▇▇▇███████████████████████████████
train_loss,█▆▅▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▂▃▄▄▅▆▆▆▇▇▇███████▇████████████████████
val_f1,▁▂▃▄▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████████████
val_loss,█▆▅▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_precision,█▂▃▂▂▁▃▃▁▁▁▂▂▂▂▂▃▃▃▂▃▃▃▃▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
val_recall,▇▁▂▃▄▄▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇████████████████
epoch,9990
train_accuracy,0.3562
train_loss,1.49748


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_accuracy,▁▁▃▄▇███████████████████████████████████
train_loss,█▆▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▃▅▆▇██████████████████████████████████
val_f1,▁▂▄▆▇███████████████████████████████████
val_loss,█▆▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_precision,▁▂█▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
val_recall,▇█▂▇▁▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▆▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆
epoch,9990
train_accuracy,0.42774
train_loss,1.25739


In [4]:
data = pd.read_csv('wineQT.csv')                                # Load the dataset
X = data.drop(['quality'], axis=1)                              # Extract features and labels
y = data['quality']
encoder = OneHotEncoder(sparse_output=False)                    # One-hot encode the labels
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))
scaler = StandardScaler()                                       # Scale the data
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


mlp = MLPClassifier(input_size=X.shape[1], hidden_layers=[10, 8], output_size=y_encoded.shape[1], learning_rate=0.01, activation='sigmoid')

mlp.train(X_train, y_train, num_epochs=5000)
predictions = mlp.predict(X_test)                                   # Predict on test data
predicted_labels = encoder.inverse_transform(predictions)           # Convert the one-hot encoded labels to categorical
y_test_labels = encoder.inverse_transform(y_test)                   # Convert the one-hot encoded labels to categorical
print("Classification Report:\n", classification_report(y_test_labels, predicted_labels, zero_division=0))

mlp = MLPClassifier(input_size=X.shape[1], hidden_layers=[10, 8], output_size=y_encoded.shape[1], learning_rate=0.01, activation='tanh')

mlp.trainbatch(X_train, y_train, num_epochs=5000)
predictions = mlp.predict(X_test)                                   # Predict on test data
predicted_labels = encoder.inverse_transform(predictions)           # Convert the one-hot encoded labels to categorical
y_test_labels = encoder.inverse_transform(y_test)                   # Convert the one-hot encoded labels to categorical
print("Classification Report:\n", classification_report(y_test_labels, predicted_labels, zero_division=0))

mlp = MLPClassifier(input_size=X.shape[1], hidden_layers=[10, 8], output_size=y_encoded.shape[1], learning_rate=0.01, activation='relu')

mlp.trainminibatch(X_train, y_train, num_epochs=5000)
predictions = mlp.predict(X_test)                                   # Predict on test data
predicted_labels = encoder.inverse_transform(predictions)           # Convert the one-hot encoded labels to categorical
y_test_labels = encoder.inverse_transform(y_test)                   # Convert the one-hot encoded labels to categorical
print("Classification Report:\n", classification_report(y_test_labels, predicted_labels, zero_division=0))

Classification Report:
               precision    recall  f1-score   support

           4       0.00      0.00      0.00         6
           5       0.68      0.70      0.69        96
           6       0.57      0.65      0.60        99
           7       0.39      0.27      0.32        26
           8       0.00      0.00      0.00         2

    accuracy                           0.60       229
   macro avg       0.33      0.32      0.32       229
weighted avg       0.58      0.60      0.59       229

Classification Report:
               precision    recall  f1-score   support

           4       0.00      0.00      0.00         6
           5       0.50      0.51      0.51        96
           6       0.49      0.59      0.53        99
           7       0.25      0.12      0.16        26
           8       0.00      0.00      0.00         2

    accuracy                           0.48       229
   macro avg       0.25      0.24      0.24       229
weighted avg       0.45      

### Clearly for most runs, we get a better accuracy compared to logistic regression but for some cases it may be worse, it depends on our choices

Classification Report:
               precision    recall  f1-score   support

           4       0.00      0.00      0.00         6
           5       0.71      0.68      0.70        96
           6       0.62      0.76      0.68        99
           7       0.59      0.38      0.47        26
           8       0.00      0.00      0.00         2

    accuracy                           0.66       229
   macro avg       0.38      0.36      0.37       229
weighted avg       0.63      0.66      0.64       229

Classification Report:
               precision    recall  f1-score   support

           4       0.00      0.00      0.00         6
           5       0.55      0.67      0.60        96
           6       0.53      0.49      0.51        99
           7       0.19      0.12      0.14        26
           8       0.00      0.00      0.00         2

    accuracy                           0.51       229
   macro avg       0.25      0.26      0.25       229
weighted avg       0.48      0.51      0.49       229

Classification Report:
               precision    recall  f1-score   support

           4       0.00      0.00      0.00         6
           5       0.64      0.71      0.67        96
           6       0.59      0.66      0.62        99
           7       0.67      0.31      0.42        26
           8       0.00      0.00      0.00         2

    accuracy                           0.62       229
   macro avg       0.38      0.33      0.34       229
weighted avg       0.60      0.62      0.60       229


# 2.2 with Power Set 2^n possibilities

In [4]:
# Learning Rate, hidden layer, epoch = constant

data = pd.read_csv('advertisement.csv')
scaler = StandardScaler()
data['age'] = scaler.fit_transform(data['age'].values.reshape(-1, 1))
data['income'] = scaler.fit_transform(data['income'].values.reshape(-1, 1))
data = pd.get_dummies(data, columns=['gender', 'education', 'married', 'city', 'occupation', 'most bought item'])
X = data.drop(['labels'], axis=1)  # Extract features
y = data['labels']
encoder = OneHotEncoder(sparse_output=False)  # Initialize the OneHotEncoder
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))  # One-hot encode the labels
scaler = StandardScaler()  # Scale the data
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
output_classes = y_encoded.shape[1]

# iterate over various learning rates, activation functions, and hidden layers and log on WandB
learning_rates = [0.001]
activations = ['tanh', 'relu', 'sigmoid']
hidden_layers = [[10, 15, 10]]

for activation in activations:
    for lr in learning_rates:
        for hidden_layer in hidden_layers:
            wandb.init(project="Ques2BB", config={"learning_rate": lr, "activation": activation, "hidden_layer": hidden_layer, "Optimizer": "SGD"})
            config = wandb.config
            mlp = MLPClassifier(X_train.shape[1], config.hidden_layer, y_encoded.shape[1], config.learning_rate, config.activation)
            mlp.train(X_train, y_train, 5000)
            predictions = mlp.predict(X_test)
            # Inverse transform the one-hot encoded labels to categorical
            predicted_labels = encoder.inverse_transform(predictions)
            y_test_labels = encoder.inverse_transform(y_test)
            accuracy =  accuracy_score(y_test_labels, predicted_labels)
            precision = precision_score(y_test_labels, predicted_labels, average='macro', zero_division=0)
            recall =  recall_score(y_test_labels, predicted_labels, average='macro', zero_division=0)
            f1 = f1_score(y_test_labels, predicted_labels, average='macro', zero_division=0)
            loss = log_loss(y_test, predictions)
            wandb.log({"val_accuracy": accuracy, "val_precision": precision, "val_recall": recall, "val_f1": f1, "val_loss": loss})

            predictions = mlp.predict(X_train)
            predicted_labels = encoder.inverse_transform(predictions)
            y_test_labels = encoder.inverse_transform(y_train)
            train_acc = accuracy_score(y_test_labels, predicted_labels)
            train_loss = log_loss(y_train, predictions)
            wandb.log({"train_accuracy": train_acc, "train_loss": train_loss})

for lr in learning_rates:
    for activation in activations:
        for hidden_layer in hidden_layers:
            wandb.init(project="Ques2BB", config={"learning_rate": lr, "activation": activation, "hidden_layer": hidden_layer, "Optimizer": "Batch"})
            config = wandb.config
            mlp = MLPClassifier(X_train.shape[1], config.hidden_layer, y_encoded.shape[1], config.learning_rate, config.activation)
            mlp.trainbatch(X_train, y_train, 5000)
            predictions = mlp.predict(X_test)
            # Inverse transform the one-hot encoded labels to categorical
            predicted_labels = encoder.inverse_transform(predictions)
            y_test_labels = encoder.inverse_transform(y_test)
            accuracy =  accuracy_score(y_test_labels, predicted_labels)
            precision = precision_score(y_test_labels, predicted_labels, average='macro', zero_division=0)
            recall =  recall_score(y_test_labels, predicted_labels, average='macro', zero_division=0)
            f1 = f1_score(y_test_labels, predicted_labels, average='macro', zero_division=0)
            loss = log_loss(y_test, predictions)
            wandb.log({"val_accuracy": accuracy, "val_precision": precision, "val_recall": recall, "val_f1": f1, "val_loss": loss})

            predictions = mlp.predict(X_train)
            predicted_labels = encoder.inverse_transform(predictions)
            y_test_labels = encoder.inverse_transform(y_train)
            train_acc = accuracy_score(y_test_labels, predicted_labels)
            train_loss = log_loss(y_train, predictions)
            wandb.log({"train_accuracy": train_acc, "train_loss": train_loss})


for lr in learning_rates:
    for activation in activations:
        for hidden_layer in hidden_layers:
            wandb.init(project="Ques2BB", config={"learning_rate": lr, "activation": activation, "hidden_layer": hidden_layer, "Optimizer": "MiniBatch"})
            config = wandb.config
            mlp = MLPClassifier(X_train.shape[1], config.hidden_layer, y_encoded.shape[1], config.learning_rate, config.activation)
            mlp.trainminibatch(X_train, y_train, 5000)
            predictions = mlp.predict(X_test)
            # Inverse transform the one-hot encoded labels to categorical
            predicted_labels = encoder.inverse_transform(predictions)
            y_test_labels = encoder.inverse_transform(y_test)
            accuracy =  accuracy_score(y_test_labels, predicted_labels)
            precision = precision_score(y_test_labels, predicted_labels, average='macro', zero_division=0)
            recall =  recall_score(y_test_labels, predicted_labels, average='macro', zero_division=0)
            f1 = f1_score(y_test_labels, predicted_labels, average='macro', zero_division=0)
            loss = log_loss(y_test, predictions)
            wandb.log({"val_accuracy": accuracy, "val_precision": precision, "val_recall": recall, "val_f1": f1, "val_loss": loss})

            predictions = mlp.predict(X_train)
            predicted_labels = encoder.inverse_transform(predictions)
            y_test_labels = encoder.inverse_transform(y_train)
            train_acc = accuracy_score(y_test_labels, predicted_labels)
            train_loss = log_loss(y_train, predictions)
            wandb.log({"train_accuracy": train_acc, "train_loss": train_loss})

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


train_accuracy,▁
train_loss,▁
val_accuracy,▁
val_f1,▁
val_loss,▁
val_precision,▁
val_recall,▁
train_accuracy,0.12875
train_loss,4.12224
val_accuracy,0.005
val_f1,0.00126


/Users/arygup/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2916: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
/Users/arygup/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2916: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


...

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


train_accuracy,▁
train_loss,▁
val_accuracy,▁
val_f1,▁
val_loss,▁
val_precision,▁
val_recall,▁
train_accuracy,0.00375
train_loss,8.2578
val_accuracy,0.0
val_f1,0.0


/Users/arygup/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2916: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


train_accuracy,▁
train_loss,▁
val_accuracy,▁
val_f1,▁
val_loss,▁
val_precision,▁
val_recall,▁
train_accuracy,0.0225
train_loss,5.77792
val_accuracy,0.025
val_f1,0.00034


In [5]:
# Optimizer, Activation = constant

data = pd.read_csv('advertisement.csv')
scaler = StandardScaler()
data['age'] = scaler.fit_transform(data['age'].values.reshape(-1, 1))
data['income'] = scaler.fit_transform(data['income'].values.reshape(-1, 1))
data = pd.get_dummies(data, columns=['gender', 'education', 'married', 'city', 'occupation', 'most bought item'])
X = data.drop(['labels'], axis=1)  # Extract features
y = data['labels']
encoder = OneHotEncoder(sparse_output=False)  # Initialize the OneHotEncoder
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))  # One-hot encode the labels
scaler = StandardScaler()  # Scale the data
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
output_classes = y_encoded.shape[1]

# iterate over various learning rates, activation functions, and hidden layers and log on WandB
learning_rates = [0.01, 0.001, 0.0001]
activations = ['sigmoid']
hidden_layers = [[10], [10, 15], [10, 15, 10]]

for lr in learning_rates:
    for activation in activations:
        for hidden_layer in hidden_layers:
            wandb.init(project="Ques2B", config={"learning_rate": lr, "activation": activation, "hidden_layer": hidden_layer, "Optimizer": "Batch"})
            config = wandb.config
            mlp = MLPClassifier(X_train.shape[1], config.hidden_layer, y_encoded.shape[1], config.learning_rate, config.activation)
            mlp.trainbatch(X_train, y_train, 1000)

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_f1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
val_precision,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_recall,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,760
train_accuracy,0.0025
train_loss,10.50383


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁███████████████████████
train_loss,██▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▅▅██████████████████████████████
val_f1,▁▁▁▁▁▁▁▁▃▃██▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
val_loss,██▇▇▇▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
val_precision,▁▁▁▁▁▁▁▁▃▃██▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
val_recall,▁▁▁▁▁▁▁▁▅▅██████████████████████████████
epoch,999
train_accuracy,0.0025
train_loss,7.75022


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


...

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██████████████████
train_loss,███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁██████████████████████████
val_f1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁██████████▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆
val_loss,███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
val_precision,▁▁▁▁▁▁▁▁▁▁▁▁▁▁█████▇▇███▇▇▇▇▆▇▇▇▆▆▆▆▆▆▆▆
val_recall,▁▁▁▁▁▁▁▁▁▁▁▁▁▁██████████████████████████
epoch,999
train_accuracy,0.0075
train_loss,6.7307


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_f1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁
val_precision,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_recall,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,999
train_accuracy,0.00125
train_loss,8.79405


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_f1,▁▂▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇██████████████████████
val_loss,███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁
val_precision,▁▂▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇██████████████████████
val_recall,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,999
train_accuracy,0.005
train_loss,8.44142


In [7]:
# Load the dataset
data = pd.read_csv('advertisement.csv')

# Standardize numerical features
scaler = StandardScaler()
data['age'] = scaler.fit_transform(data['age'].values.reshape(-1, 1))
data['income'] = scaler.fit_transform(data['income'].values.reshape(-1, 1))

# Perform one-hot encoding for categorical features
data = pd.get_dummies(data, columns=['gender', 'education', 'married', 'city', 'occupation', 'most bought item'])

X = data.drop(['labels'], axis=1)  # Extract features
y = data['labels']

encoder = OneHotEncoder(sparse_output=False)  # Initialize the OneHotEncoder
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))  # One-hot encode the labels

scaler = StandardScaler()  # Scale the data
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Specify the number of classes for the output layer
output_classes = y_encoded.shape[1]

mlp = MLPClassifier(input_size=X.shape[1], hidden_layers=[10, 15], output_size=output_classes, learning_rate=0.01, activation='relu')       # best performing

mlp.trainminibatch(X_train, y_train, num_epochs=1200)

predictions = mlp.predict(X_test)
# Inverse transform the one-hot encoded labels to categorical
predicted_labels = encoder.inverse_transform(predictions)
y_test_labels = encoder.inverse_transform(y_test)
print("Classification Report:\n", classification_report(y_test_labels, predicted_labels, zero_division=0))


Classification Report:
                                                     precision    recall  f1-score   support

                                            beauty       0.00      0.00      0.00         4
                                      beauty books       0.00      0.00      0.00         1
                                   beauty clothing       0.00      0.00      0.00         2
                              beauty clothing food       0.00      0.00      0.00         2
                              beauty clothing home       0.00      0.00      0.00         1
                                beauty electronics       0.00      0.00      0.00         1
                          beauty electronics books       0.00      0.00      0.00         1
                       beauty electronics clothing       0.00      0.00      0.00         2
                      beauty electronics food home       0.00      0.00      0.00         1
                      beauty electronics furniture     